In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from config import DOWNLOAD_DIR
from pathlib import Path


### Merge datasets, create daily sum of total precipitation

We need to merge the three (sometimes two?) datasets into a single dataframe.  Then, we need to produce total sums for each day.  Source data will contain a [mix of ERA5 and ERA5T data](https://confluence.ecmwf.int/display/CUSF/ERA5+CDS+requests+which+return+a+mixture+of+ERA5+and+ERA5T+data), so we need to do this to get a consistent set of data:
 
### Our algorithm for merging/fusing data needs to go like this,


 1. Some data files may not have `expver` present at all, such as older data which are outside the window of modification for the rolling ERA5 updates.  Use those data as-is.
 1. If `expver` has both values 1 and 5 (a mix of ERA5 and ERA5T data), choose `expver == 1` where possible, and `expver == 5` for timesteps when `expver == 1` is missing.
 1. For recent data, only `expver` value 5 is present.

#### Notes/Questions?

 * Current month may be only ERA5T / `expver = 5`
 * Prior months in the same year may have mix of `expver` 5 and 1.
 * Prior year doesn't have `expver`.


Needs to be robust to the presence/absence of expver.

Goal is that a dataframe taken for a single point looks correct with data for every timestep (no dupes, no missing).

## TODO: make this work properly for January when we may only have 2 data files.

In [2]:
prior_year = xr.open_dataset(Path(DOWNLOAD_DIR) / "inputs" / "total_precipitation_previous_year.nc")
current_year = xr.open_dataset(Path(DOWNLOAD_DIR) / "inputs" / "total_precipitation_current_year.nc")
current_month = xr.open_dataset(Path(DOWNLOAD_DIR) / "inputs" / "total_precipitation_current_month.nc")

FileNotFoundError: [Errno 2] No such file or directory: b'/tmp/nws_drought/inputs/total_precipitation_previous_year.nc'